Initialize imports

In [1]:
from IPython.display import JSON

import ipywidgets as widgets
from ipywidgets import fixed, interact

from openfisca_core import decompositions, periods
from openfisca_france import FranceTaxBenefitSystem
from openfisca_matplotlib import graphs, utils

import matplotlib.pyplot as plt
%matplotlib inline

## Initialize tax and benefit system of France

In [2]:
tbs = FranceTaxBenefitSystem()

## Precalculate "revenu disponible"

Define some helpers:

In [67]:
def count_to_step(min, max, count):
    """Examples:
    >>> count_to_step(0, 80, 5)
    20
    """
    return float(max - min) / (count - 1)

def value_to_index(min, step, value):
    """Examples:
    >>> value_to_index(0, 10, 0)
    0
    >>> value_to_index(0, 10, 40)
    4
    >>> value_to_index(3, 1, 6)
    3
    """
    return int((value / step) - min)

In [46]:
def precalculate_revenu_disponible(period, min, max, count):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": 30,
        },
        "axes": [
            dict(
                count = count,
                min = min,
                max = max,
                name = 'salaire_de_base',
            ),
        ],
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    revenu_disponible = simulation.calculate("revenu_disponible", period)
    return revenu_disponible

In [81]:
count = 1000
min = 0
max = 500000
step = count_to_step(min, max, count)
initial_value = 0

In [85]:
revenu_disponible = precalculate_revenu_disponible(periods.period("2015"), min, max, count)

## Display "Revenu disponible" interactively

In [86]:
def display_revenu_disponible(salaire_de_base, revenu_disponible, min, step):
    index = value_to_index(min, step, salaire_de_base)
    return revenu_disponible[index]

In [87]:
interact(
    display_revenu_disponible,
    salaire_de_base=widgets.FloatSlider(min=min, max=max, step=step, value=initial_value),
    revenu_disponible=fixed(revenu_disponible),
    min=fixed(min),
    step=fixed(step),
)
None  # Hide strange output

interactive(children=(FloatSlider(value=0.0, description=u'salaire_de_base', max=500000.0, step=500.5005005005005), Output()), _dom_classes=('widget-interact',))

## Display interactive waterfall

In [90]:
def precalculate_waterfall(min, max, count, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": 30,
        },
        "axes": [
            dict(
                count = count,
                min = min,
                max = max,
                name = 'salaire_de_base',
            ),
        ],
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    decomposition_json = decompositions.get_decomposition_json(tbs)
    decomposition_json_precalculated = decompositions.calculate([simulation], decomposition_json)
    return decomposition_json_precalculated

Define some helpers:

In [93]:
def update_key(mapping, key, value):
    return {
        k: value if k == key else v
        for k, v in mapping.iteritems()
    }

def keep_value_at_index(index, node):
    """Transform decomposition JSON tree, keeping only the given index for each node["values"]."""
    if "values" in node:
        node = update_key(node, "values", [node["values"][index]])
    if "children" in node:
        new_children = [
            keep_value_at_index(index, child_node)
            for child_node in node["children"]
        ]
        node = update_key(node, "children", new_children)
    return node

In [91]:
decomposition_json_precalculated = precalculate_waterfall(min, max, count, periods.period("2015"))

<IPython.core.display.JSON object>

In [95]:
def display_precalculated_waterfall(salaire_de_base, min, step):
    index = value_to_index(min, step, salaire_de_base)
    out_node = utils.OutNode()
    utils.convert_to_out_node(out_node, keep_value_at_index(index, decomposition_json_precalculated))
    out_node.setLeavesVisible()
    fig = plt.figure()
    axes = fig.gca()
    graphs.draw_waterfall_from_node_data(out_node, axes, tbs.CURRENCY)

In [96]:
interact(
    display_precalculated_waterfall,
    salaire_de_base=widgets.FloatSlider(min=min, max=max, step=step, value=initial_value),
    min=fixed(min),
    step=fixed(step),
)
None  # Hide strange output

interactive(children=(FloatSlider(value=0.0, description=u'salaire_de_base', max=500000.0, step=500.5005005005005), Output()), _dom_classes=('widget-interact',))

## Display bareme

Instead of varying the salary with a slider, let's climb on the [ladder of abstraction](http://worrydream.com/LadderOfAbstraction/) to see *all* the possible waterfall charts at a time. This new chart is called a "bareme" in French.

In [99]:
def display_bareme(age, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": age,
        },
        "axes": [
            dict(
                count = 10,
                min = 0,
                max = 50000,
                name = 'salaire_de_base',
            ),
        ],
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    graphs.draw_bareme(
        simulation = simulation,
        x_axis = "salaire_imposable",
        legend_position = 2,
        bbox_to_anchor = (1.05, 1),
    )

We added a slider to change "age", but changing it will trigger a new computation, potentially slow.

For example, you can set the age to "80" and see "Allocation de solidarité aux personnes agées" appear.

In [100]:
interact(
    display_bareme,
    age=widgets.IntSlider(min=0, max=130, step=1, value=30, continuous_update=False),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(IntSlider(value=30, continuous_update=False, description=u'age', max=130), Output()), _dom_classes=('widget-interact',))